In [30]:
import torch
import numpy as np

## First, let's create the NN using numpy

N = batch size, D is input dimension
H is hidden dimension, D_out is output dimension

In [31]:
N, D_in, H, D_out = 64, 1000, 100, 10

Let's create some input and output data.

In [32]:
# 10 labels for each 64x1000 instances
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

Randomly initialize the weights.

In [33]:
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6

In [34]:
for t in range(500):
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)
    
    # Compute the loss using RSS
    loss = np.square(y_pred-y).sum()
    if (t%100 == 0):
        print("loss at iteration ", t, ": ", loss)
    
    # Back propogate to compute the gradients with resepct to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h<0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

loss at iteration  0 :  24248677.38877599
loss at iteration  100 :  394.43101116373657
loss at iteration  200 :  2.0939072931860077
loss at iteration  300 :  0.021066061613267635
loss at iteration  400 :  0.00025531930443143405


## Now we can do the same thing but using torch.

In [35]:
torch.cuda.is_available()

True

In [36]:
dtype = torch.float
device = torch.device("cpu")
device = torch.device("cuda")

# Random numbers.
N, D_in, H, D_out = 64, 1000, 100, 10

Create random input and output data.

In [37]:
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

Randomly init the weights

In [38]:
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

learning_rate=1e-6

In [39]:
import time
first = time.time()
for t in range(50000):
    # Forward pass 
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # Compute loss
    loss = (y_pred - y).pow(2).sum()
    if t % 10000 == 0:
        print("loss at iteration ", t , " : " , loss.item())
    
    # Backprop to compute gradients - autograd will compute the gradient of loss
    # with resepct to all tensors with requires_grad=True.
    # w1.grad and w2.grad will be tensors holding the gradient of the loss wrt to w1 w2
    loss.backward()
    
    # Update the weights using gradient descent. wrap in torch.no_grad() because 
    # weights have requires_grad=True; but we don't need to track this. 
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # set the gradients back to zero 
        w1.grad.zero_()
        w2.grad.zero_()
end = time.time()
print(end-first)

loss at iteration  0  :  28006882.0
loss at iteration  10000  :  1.6635647170915036e-07
loss at iteration  20000  :  8.044061416967452e-08
loss at iteration  30000  :  6.056331613990551e-08
loss at iteration  40000  :  5.100554290038417e-08
29.51629137992859
